# Import

In [15]:
import pandas as pd
import numpy as np
import requests
import csv
import pandas as pd
import urllib.parse
import urllib.request
from bs4 import BeautifulSoup
from numpy import loadtxt,savetxt


# Ratings with Timestamp

In [2]:
r_cols = ['userId', 'movieId', 'rating', 'timestamp']
ratings_base = pd.read_csv('./ml-1m/ratings.dat',sep='::', names=r_cols,engine='python')

In [3]:
ratings_base

,userId,movieId,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [4]:
ratings_base['timestamp'] = pd.to_datetime(ratings_base['timestamp'],unit='s').apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))

In [5]:
ratings_base

,userId,movieId,rating,timestamp
0,1,1193,5,2000-12-31 22:12:40
1,1,661,3,2000-12-31 22:35:09
2,1,914,3,2000-12-31 22:32:48
3,1,3408,4,2000-12-31 22:04:35
4,1,2355,5,2001-01-06 23:38:11
...,...,...,...,...
1000204,6040,1091,1,2000-04-26 02:35:41
1000205,6040,1094,5,2000-04-25 23:21:27
1000206,6040,562,5,2000-04-25 23:19:06
1000207,6040,1096,4,2000-04-26 02:20:48


In [6]:
ratings_base = np.asmatrix(ratings_base)
for i in range(ratings_base.shape[0]):
    if ratings_base[i,3] != '1970-01-01 00:00:00':
        ratings_base[i,3] = '1970-01-01 00:00:00'

In [7]:
ratings_base = pd.DataFrame(ratings_base, columns = ['userId', 'movieId', 'rating', 'timestamp'])

In [8]:
ratings_base.to_csv(r'./data/ratings.csv', index=True)

# Movies with Posters
https://github.com/babu-thomas/movielens-posters/issues/1

In [ ]:
df = pd.read_csv('movies.dat', delimiter=',', header=None, 
    names=['movie_id', 'title', 'genre'], engine='python')

for i, row in df.iterrows():
    movie_id = row['movie_id']
    movie_title = row['title']
    domain = 'http://www.imdb.com'
    search_url = domain + '/find?q=' + urllib.parse.quote_plus(movie_title)
    if movie_id <= 2347:
        pass
    with urllib.request.urlopen(search_url) as response:
        html = response.read()
        soup = BeautifulSoup(html, 'html.parser')
        # Get url of 1st search result
        try:
            title = soup.find('table', class_='findList').tr.a['href']
            movie_url = domain + title
            with open('movie_url.csv', 'a', newline='') as out_csv:
                writer = csv.writer(out_csv, delimiter=',')
                writer.writerow([movie_id, movie_url])
        # Ignore cases where search returns no results
        except AttributeError:
            pass

In [ ]:
start = time.time()
print('start', start)
row_names = ['movie_id', 'movie_url']
with open('movie_url.csv', 'r', newline='') as in_csv:
    reader = csv.DictReader(in_csv, fieldnames=row_names, delimiter=',')

    for row in reader:
        movie_id = row['movie_id']
        movie_url = row['movie_url']
        domain = 'http://www.imdb.com'

        with urllib.request.urlopen(movie_url) as response:
            html = response.read()
            soup = BeautifulSoup(html, 'html.parser')
            # Get url of poster image
            try:
                image_url = soup.find('div', class_='poster').a.img['src']
                extension = image_url[-4:]
                image_url = ''.join(image_url.partition('_')[0]) + extension
                filename = 'img/' + movie_id + extension
                with urllib.request.urlopen(image_url) as response:
                    with open('movie_poster.csv', 'a', newline='') as out_csv:
                        writer = csv.writer(out_csv, delimiter=',')
                        writer.writerow([movie_id, image_url])
            # Ignore cases where no poster image is present
            except AttributeError:
                pass
        print('movie_id:', movie_id)
print('time:', time.time()-start)

In [ ]:
i_cols = ['movie id', 'movie title', 'movie genre']
movies = pd.read_csv('ml-1m\movies.dat', sep='::', names=i_cols, encoding='latin-1')
movies = np.asmatrix(movies)
movies = np.append(movies, np.zeros(movies.shape[0]).reshape(3883,1), axis=1)
moviePosters = pd.read_csv('movie_poster.csv', sep=',', names=["1","2"], encoding='latin-1')
moviePosters = np.asmatrix(moviePosters)
for i in range(movies.shape[0]):
    for j in range(moviePosters.shape[0]):
        if movies[i,0] == moviePosters[j,0]:
            movies[i,3] = moviePosters[j,1]

In [111]:
r_cols = ['movieId', 'movieTitle', 'movieGenre', 'moviePoster']
movies = pd.read_csv('movies.csv',sep=',', names=r_cols,engine='python')
movies = np.asmatrix(movies)
for i in range(movies.shape[0]):
    if ', The ' in movies[i,1]:
        indexComma = movies[i,1].index(', The')
        movies[i,1] = movies[i,1][indexComma+2:indexComma+5] +" "+ movies[i,1][0:indexComma]+" " + movies[i,1][indexComma+6:]
for i in range(movies.shape[0]):
    if ', An ' in movies[i,1]:
        indexComma = movies[i,1].index(', An')
        movies[i,1] = movies[i,1][indexComma+2:indexComma+4] +" "+ movies[i,1][0:indexComma]+" " + movies[i,1][indexComma+5:]
for i in range(movies.shape[0]):
    if ', A ' in movies[i,1]:
        indexComma = movies[i,1].index(', A ')
        movies[i,1] = movies[i,1][indexComma+2:indexComma+3] +" "+ movies[i,1][0:indexComma]+" " + movies[i,1][indexComma+4:]
moviesPD = pd.DataFrame(movies)
moviesPD.to_csv(r'moviesWithPosters.csv', index=False)

In [136]:
r_cols = ['movieId', 'movieTitle', 'movieGenre', 'moviePoster']
movies = pd.read_csv('moviesWithPosters.csv',sep=',', names=r_cols,engine='python')
movies.to_csv(r'indexmoviesWithPosters.csv', index=True)

# Export to CSV MFCF Model

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [10]:
class MF(object):
    def __init__(self, Y, K, lam = 0.1, Xinit = None, Winit = None, 
    learning_rate = 0.5, max_iter = 10000, print_every = 100):
        self.Y = Y      # represents the utility matrix
        self.K = K      # number of features
        self.lam = lam  # regularization parameter
        self.learning_rate = learning_rate  # for gradient descent
        self.max_iter = max_iter            # maximum number of iterations
        self.print_every = print_every      # print loss after each a few iters
        self.n_users = int(np.max(Y[:, 0])) + 1
        self.users_ids = np.unique(np.asarray(Y[:,0].reshape(Y[:,0].shape[0]))).astype(np.int32)
        self.n_items = int(np.max(Y[:, 1])) + 1
        self.items_ids = np.unique(np.asarray(Y[:,1].reshape(Y[:,1].shape[0]))).astype(np.int32)
        self.n_ratings = Y.shape[0]
        self.X = np.random.randn(self.n_items, K) if Xinit is None else Xinit
        self.W = np.random.randn(K, self.n_users) if Winit is None else Winit
        self.b = np.random.randn(self.n_items)  # item biases
        self.d = np.random.randn(self.n_users)  # user biases

    # return current loss value
    def loss(self):
        L = 0
        for i in range(self.n_ratings):
            # user_id, item_id, rating
            n, m, rating = int(self.Y[i, 0]), int(self.Y[i, 1]), self.Y[i, 2]
            L += 0.5*(self.X[m].dot(self.W[:,n]) + self.b[m] + self.d[n] - rating)**2
        L /= self.n_ratings
        # regularization, don't ever forget this
        return L + 0.5*self.lam*(np.sum(self.X**2) + np.sum(self.W**2))
    
    def updateXb(self):
        for m in self.items_ids:
            ids = np.where(self.Y[:, 1] == m)[0] # row indices of items m
            user_ids, ratings = self.Y[ids, 0].astype(np.int32), self.Y[ids, 2]
            Wm, dm = self.W[:, user_ids], self.d[user_ids]
            Wm = Wm.reshape(Wm.shape[0], Wm.shape[1])
            for i in range(30): # 30 iteration for each sub problem
                xm = self.X[m]
                error = Wm.T.dot(xm).reshape(-1,1) + self.b[m] + dm - ratings
                grad_xm = Wm.dot(error)/self.n_ratings + (self.lam*xm).reshape(-1,1)
                grad_bm = np.sum(error)/self.n_ratings
                # gradient descent
                self.X[m] -= np.array((self.learning_rate*grad_xm).T)[0]
                self.b[m] -= self.learning_rate*grad_bm

    def updateWd(self):
        for n in self.users_ids:
            # get all items rated by user n, and the corresponding ratings
            ids = np.where(self.Y[:, 0] == n)[0]
            item_ids, ratings = self.Y[ids, 1].astype(np.int32), self.Y[ids, 2]
            Xn, bn = self.X[item_ids], self.b[item_ids]
            for i in range(30): # 30 iteration for each sub problem
                wn = self.W[:, n]
                error = Xn.dot(wn) + bn + self.d[n] - ratings
                grad_wn = Xn.T.dot(error)/self.n_ratings + self.lam*wn
                grad_dn = np.sum(error)/self.n_ratings
                # gradient descent
                self.W[:, n] -= np.array(self.learning_rate*grad_wn.reshape(-1))[0]
                self.d[n]    -= self.learning_rate*grad_dn

    def updateXbItemI(self, itemID):
        m = itemID
        ids = np.where(self.Y[:, 1] == m)[0] # row indices of items m
        user_ids, ratings = self.Y[ids, 0].astype(np.int32), self.Y[ids, 2]
        Wm, dm = self.W[:, user_ids], self.d[user_ids]
        Wm = Wm.reshape(Wm.shape[0], Wm.shape[1])
        for i in range(30): # 30 iteration for each sub problem
            xm = self.X[m]
            error = Wm.T.dot(xm).reshape(-1,1) + self.b[m] + dm - ratings
            grad_xm = Wm.dot(error)/self.n_ratings + (self.lam*xm).reshape(-1,1)
            grad_bm = np.sum(error)/self.n_ratings
            # gradient descent
            self.X[m] -= np.array((self.learning_rate*grad_xm).T)[0]
            self.b[m] -= self.learning_rate*grad_bm

    def updateWdUserU(self, userID):
        n = userID
        # get all items rated by user n, and the corresponding ratings
        ids = np.where(self.Y[:, 0] == n)[0]
        item_ids, ratings = self.Y[ids, 1].astype(np.int32), self.Y[ids, 2]
        Xn, bn = self.X[item_ids], self.b[item_ids]
        for i in range(30): # 30 iteration for each sub problem
            wn = self.W[:, n]
            error = Xn.dot(wn) + bn + self.d[n] - ratings
            grad_wn = Xn.T.dot(error)/self.n_ratings + self.lam*wn
            grad_dn = np.sum(error)/self.n_ratings
            # gradient descent
            self.W[:, n] -= np.array(self.learning_rate*grad_wn.reshape(-1))[0]
            self.d[n]    -= self.learning_rate*grad_dn

    def fit(self):
        for it in range(self.max_iter):
            self.updateXb()
            self.updateWd()
            # if (it+1) % self.print_every == 0:
            #     rsme_train = self.evaluate_RMSE(self.Y)
            #     # print("iter = ",it+1 ,", loss = "+self.loss(),", RMSE train = ",rsme_train)
            #     print(it+1)
            #     print(self.loss())
            #     print(rsme_train)
            #     print(self.evaluate_RMSE(rate_test))

    def pred(self, u, i):
        # predict the rating of user u for item i
        u, i = int(u), int(i)
        try:
            pred = self.X[i, :].dot(self.W[:, u]) + self.b[i] + self.d[u]
        except:
            return 0
        return max(0, min(5, pred))

    def evaluate_RMSE(self, rate_test):
        n_tests = rate_test.shape[0]
        SE = 0
        for n in range(n_tests):
            pred = self.pred(rate_test[n, 0], rate_test[n, 1])
            SE += (pred - rate_test[n, 2])**2
        
        RMSE = np.sqrt(SE/n_tests)
        return RMSE
    
    def predTopTen(self, user_id):
        # predict the top 10 items for user user_id
        items_ids = np.unique(np.asarray(self.Y[:,1].reshape(self.Y[:,1].shape[0]))).astype(np.int32)
        ids = np.where(self.Y[:, 0] == user_id)[0]
        rated_item_ids = self.Y[ids, 1].astype(np.int32)
        unrated_item_ids = [x for x in items_ids if x not in rated_item_ids]
        items_ids = unrated_item_ids
        predForUserU = np.random.randn(len(items_ids), 2)
        for i in range(len(items_ids)):
            predForUserU[i] = [items_ids[i],self.pred(u= user_id,i = i)]
        predForUserU= predForUserU[predForUserU[:,1].argsort()][::-1]
        idsTopTen = predForUserU[:10,0].astype(np.int32).tolist()
        ratingsTopTen = predForUserU[:10,1].tolist()
        return(idsTopTen,ratingsTopTen)

In [11]:
i_cols = ['userIdd', 'movieId', 'rating','timestamp']
ratings = pd.read_csv('./ml-1m/ratings.dat', sep='::', names=i_cols, encoding='latin-1', engine='python')
ratings_matrix = rate_train = np.asmatrix(ratings)
ratings_matrix[:, :2] -= 1
rate_train, rate_test = train_test_split(ratings_matrix, test_size=0.2, random_state=10)

In [18]:
rs = MF(rate_train, K = 50, lam = .01, print_every = 1, learning_rate = 50,max_iter = 30)
# rs.fit()
# To load X W b d
XX = loadtxt('./model/XX.csv', delimiter=',')
WW = loadtxt('./model/WW.csv', delimiter=',')
bb = loadtxt('./model/bb.csv', delimiter=',')
dd = loadtxt('./model/dd.csv', delimiter=',')
rs.X = XX
rs.W = WW
rs.b = bb
rs.d = dd
rs.evaluate_RMSE(rate_test)

0.9217762262677203

In [ ]:
# To save X W b d
# XX = rs.X
# WW = rs.W.T
# bb = rs.b
# dd = rs.d
# savetxt('XX.csv', XX, delimiter=',')
# savetxt('WW.csv', WW, delimiter=',')
# savetxt('bb.csv', bb, delimiter=',')
# savetxt('dd.csv', dd, delimiter=',')

In [20]:
XXI = pd.DataFrame(XX, columns = ['e1','e2','e3','e4','e5','e6','e7','e8','e9','e10','e11','e12','e13','e14','e15','e16','e17','e18','e19','e20','e21','e22','e23','e24','e25','e26','e27','e28','e29','e30','e31','e32','e33','e34','e35','e36','e37','e38','e39','e40','e41','e42','e43','e44','e45','e46','e47','e48','e49','e50'])
WWI = pd.DataFrame(WW.T, columns = ['e1','e2','e3','e4','e5','e6','e7','e8','e9','e10','e11','e12','e13','e14','e15','e16','e17','e18','e19','e20','e21','e22','e23','e24','e25','e26','e27','e28','e29','e30','e31','e32','e33','e34','e35','e36','e37','e38','e39','e40','e41','e42','e43','e44','e45','e46','e47','e48','e49','e50'])
bbI = pd.DataFrame(bb, columns = ['e1'])
ddI = pd.DataFrame(dd, columns = ['e1'])

In [21]:
XXI.to_csv(r'./data/XXI.csv', index=True)
WWI.to_csv(r'./data/WWI.csv', index=True)
bbI.to_csv(r'./data/bbI.csv', index=True)
ddI.to_csv(r'./data/ddI.csv', index=True)